In [6]:
import os,sys
project_root = os.path.abspath(os.path.join(os.getcwd(),".."))

if project_root  not in sys.path:
    sys.path.insert(0,project_root)

from utils import create_spark_session
from schemas import cancer_schema